## Classical SDP relaxation of OPF
As a first step, we present a simple implementation of the classical SDP formulation of the Optimnal Power Flow problem using CVXPY.

### Setup
We start by importing pandapower for the test cases;
CVXPY, an open source Python-embedded modeling language for convex optimization problems;
MOSEK, a large scale optimization software. 

In [26]:
import pandapower.networks as nw
import pandapower as pp
import numpy as np
import cvxpy as cp
from scipy.sparse import csr_matrix
import mosek
import plotly.express as px

### Load Network Data
Now,let's make a function that loads the IEEE test cases from Pandapower.

In [2]:
# Select IEEE test case
def load_network(case):
    if case == 14:
        return nw.case14()
    elif case == 30:
        return nw.case30()
    elif case == 57:
        return nw.case57()
    elif case == 118:
        return nw.case118()
    else:
        raise ValueError("Unsupported test case")

In [3]:
# Load the chosen IEEE test case
case_number = 14 # Change to 30, 57, or 118 for larger cases
net = load_network(case_number)

pp.runpp(net)  # Run power flow to initialize values
net

This pandapower network includes the following parameter tables:
   - bus (14 elements)
   - load (11 elements)
   - gen (4 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (15 elements)
   - trafo (5 elements)
   - poly_cost (5 elements)
   - bus_geodata (14 elements)
 and the following results tables:
   - res_bus (14 elements)
   - res_line (15 elements)
   - res_trafo (5 elements)
   - res_ext_grid (1 element)
   - res_load (11 elements)
   - res_shunt (1 element)
   - res_gen (4 elements)

In [4]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,20.0,0.043029,0.0,0.0,0.0
1,0,gen,0.0,20.0,0.250000,0.0,0.0,0.0
2,1,gen,0.0,40.0,0.010000,0.0,0.0,0.0
3,2,gen,0.0,40.0,0.010000,0.0,0.0,0.0
4,3,gen,0.0,40.0,0.010000,0.0,0.0,0.0


When a power flow is carried out, the element based grid model is translated into a bus-branch model. That bus-branch model is stored in a data structure that is based on the PYPOWER/MATPOWER casefile (with some extensions). This ppc can be accessed after power flow using net._ppc
We will get the bus admittance matrix using exactly that and extract all the parameters we need.

In [5]:
#Gneration bus information
net.bus

,name,vn_kv,type,zone,in_service,max_vm_pu,min_vm_pu
0,1,135.000,b,1.0,True,1.06,0.94
1,2,135.000,b,1.0,True,1.06,0.94
2,3,135.000,b,1.0,True,1.06,0.94
3,4,135.000,b,1.0,True,1.06,0.94
4,5,135.000,b,1.0,True,1.06,0.94
5,6,0.208,b,1.0,True,1.06,0.94
6,7,14.000,b,1.0,True,1.06,0.94
7,8,12.000,b,1.0,True,1.06,0.94
8,9,0.208,b,1.0,True,1.06,0.94
9,10,0.208,b,1.0,True,1.06,0.94


In [6]:

#make sure all units are consistent (convert everything to per unit)
Sbase = net.sn_mva # unit value for power MVA
Vbase = 135 # unit value for voltage kV
Zbase = Vbase**2 / Sbase # base impedance in Ohms
Ybase = 1 / Zbase # base admittance in S

#Now, we extract network parameters
Ybus = net._ppc["internal"]["Ybus"].todense() # Bus admittance matrix (Already in per unit. We need todense since the matrices are stored as sparcse matrices, but dense are easier to work with)
n = len(net.bus)  # Number of buses
# slack_bus = net.gen.bus.values[0]  # Index of the slack bus
branch = net.line  # Get each transmission line's specification for line constraints
#we need to add line flow constraints

# Extract generator information for generation constraints
gen_buses = net.gen["bus"].values #index of the generators.
min_p_mw = net.gen["min_p_mw"].values #minimal real power
max_p_mw = net.gen["max_p_mw"].values
min_q_mw = net.gen["min_q_mvar"].values #minimal reactive power
max_q_mw = net.gen["max_q_mvar"].values

# Extract load buses for generation constraints
load_buses = net.load["bus"].values #index of the load buses.
#load active and reactive power for load
p_load = np.zeros(n)
q_load = np.zeros(n)
for i,row in net.load.iterrows():
    p_load[row['bus']] = -row['p_mw'] #negative sign because pandapower defines load as positive, but we need it as negative for consumption
    q_load[row['bus']] = -row['q_mvar']

# Voltage limits
min_v_pu = net.bus["min_vm_pu"].values ** 2  # Squared for SDP
max_v_pu = net.bus["max_vm_pu"].values ** 2

# Extract generator cost coefficients from net.poly_cost
cost = net.poly_cost.drop([0])
c2 = cost["cp2_eur_per_mw2"].values   # Quadratic cost coefficients
c1 = cost["cp1_eur_per_mw"].values   # Linear cost coefficients
c0 = cost["cp0_eur"].values         # Constant cost coefficients

# Ensure the cost coefficients have the correct size
if len(c2) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
elif len(c1) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
elif len(c0) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
else:
    print("The number of generators matches the number of cost coefficients")


# # Make the cost matrix
# C = np.zeros((n, n))
# for i, gen_bus in enumerate(gen_buses):
#     C[gen_bus, gen_bus] = c2[i]

# # Print the cost matrix
# np.set_printoptions(precision=10, suppress=True)
# print(C)

The number of generators matches the number of cost coefficients


### Define Variables and the objective function
The problem we are trying to solve in SDP OPF is
$$min \space \space trCW$$
subjected to $W \succcurlyeq 0$ and other linear constraints, which are

**Active Power Flow**
$$p_{min, j} \le tr (\Phi_jW_B) \le p_{max, j}$$
where $\Phi_j = \frac{1}{2}(Y^H_j + Y_j)$

**Reactive Power Flow**
$$q_{min, j} \le tr (\Psi_jW_B) \le q_{max, j}$$
where $\Psi_j = \frac{1}{2i}(Y^H_j + Y_j)$

**Voltage Magnitude Constraints**
$$v_{min, j} \le tr (J_jW_B) \le v_{max, j}$$
where J is the Hermitian matrix with a single 1 in the (j,j)th entry and zero everywhere else.

Now, we won't directly use Low's objective function, $tr(CW)$, because while it is elegant, this form is linear in W, so it can only express linear objectives such as 
$$ \sum_jc_{1j}Re(tr(\phi_jW)) = Re(tr(CW)) $$
But quadratic cost requires adding a coefficient to power squared, which means we have to find C such as 
$$(tr(\phi_jW))^2 = tr(CW)$$
Such C doesn't exist. Instead, we will use the following equivalent objective function:
$$\sum_{j \in \mathcal{G}} ( c_{2j} tr(\Phi_j W)^2 + c_{1j} tr(\Phi_j W))$$

In [7]:
# Define an arbitrary n x n Hermitian W (Directly enforcing W= V V^+ will require X to be rank-1, which is too strict. We use the power balance, PSD and voltage constraints to guide this arbitrary X to X = VV^+)
W = cp.Variable((n, n), hermitian=True)

# Define phi and psi matrices for each bus as the constraints
Phi = [] # Hermitian part of Y_j
Psi = [] # Skewed Hermitian part of Y_j
J = [] # Hermitian matrix with a single 1 in the (j,j)th entry and zero everywhere else.
for j in range(n):
    Y_j = np.zeros_like(Ybus,dtype=complex) #Y_j is the admittance matrix with only the j-th row, everything else is zero
    Y_j[j,:] = Ybus[j,:]
    Y_j_H = np.conj(Y_j).T #Hermitian of Y_j
    
    J_j= np.zeros_like(Ybus,dtype=complex)
    J_j[j,j] = 1

    Phi_j = 1/2 * (Y_j_H + Y_j) 
    Psi_j = 1/(2j) * (Y_j_H - Y_j)

    Phi.append(Phi_j) 
    Psi.append(Psi_j)
    J.append(J_j)


# # Objective Function, tr(CW), which should be real since cost is real
# #Even though the trace of the product of two Hermitian matrices is theoretically real, numerical errors during computation can introduce small imaginary components.
# # Explicitly using cp.real() ensures that the solver only considers the real part of the objective function.
# objective_function = cp.real(cp.trace(C @ W)) 
# objective = cp.Minimize(objective_function)

# Define the objective function, which is the sum of the costs of all generators
objective_terms = cp.sum([
    c2[i] * (cp.real(cp.trace(Phi[gen_buses[i]] @ W))*Sbase)**2 + 
    c1[i] * cp.real(cp.trace(Phi[gen_buses[i]] @ W))*Sbase for i in range(len(gen_buses))])
objective = cp.Minimize(objective_terms)

# Constraints
constraints = []
# Positive semidefinite constraint
constraints.append(W >> 0)


for j in range(n):

    #Voltage magnitude constraints
    constraints.append(min_v_pu[j]<=cp.real(cp.trace(J[j] @ W)))
    constraints.append(cp.real(cp.trace(J[j] @ W))<=max_v_pu[j])

    # Power balance equations (real and reactive)
    Pin_j = cp.trace(Phi[j]@W)*Sbase
    Qin_j = cp.trace(Psi[j]@W)*Sbase

    # If bus is a generator bus, enforce generation limits
    if j in gen_buses:
        idx = np.where(gen_buses == j)[0][0]  # Get generator index
        constraints.append(min_p_mw[idx] <= cp.real(Pin_j))
        constraints.append(cp.real(Pin_j) <= max_p_mw[idx])
        constraints.append(min_q_mw[idx] <= cp.real(Qin_j))
        constraints.append(cp.real(Qin_j) <= max_q_mw[idx])
    else:
        # For non-generator buses, enforce power balance constraints
        constraints.append(cp.real(Pin_j) == p_load[j])
        constraints.append(cp.real(Qin_j) == q_load[j])


### Solve using cvxpy's SDP solver

In [39]:
# Solve the SDP
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)

# Print constraints
print("p min (MW):", min_p_mw)
print("p max:(MW)", max_p_mw)
print("q min (MW):", min_q_mw)
print("q max (MW):", max_q_mw)

# Print results
print("Problem Status:", prob.status)
print("Optimal cost EUR:", prob.value)
P_g_sdp = [cp.real(cp.trace(Phi[j] @ W)).value * Sbase for j in gen_buses]
print("Real power generation in MW (SDP OPF):", P_g_sdp)
Q_g_sdp = [cp.imag(cp.trace(Psi[j] @ W)).value * Sbase for j in gen_buses]
print("Reactive power generation in MW (SDP OPF):", Q_g_sdp)
V_sdp = [np.sqrt(cp.real(W[j, j]).value) for j in range(n)]
print("Voltage magnitudes in pu(SDP OPF):", V_sdp)

p min (MW): [0. 0. 0. 0.]
p max:(MW) [140. 100. 100. 100.]
q min (MW): [-40.   0.  -6.  -6.]
q max (MW): [50. 40. 24. 24.]
Problem Status: optimal
Optimal cost EUR: 4952.720791188003
Real power generation in MW (SDP OPF): [40.41219018940754, 12.058095249736045, 29.91084561478661, 50.53730788733276]
Reactive power generation in MW (SDP OPF): [-4.440892098500626e-14, 0.0, 2.220446049250313e-14, 0.0]
Voltage magnitudes in pu(SDP OPF): [1.0363585957488508, 1.036993105965024, 1.0378754286990761, 1.0255319148367943, 1.023895748186668, 1.059999384990888, 1.0495108281887249, 1.0599999148382895, 1.046504039856579, 1.0413539241714243, 1.0470260831722455, 1.0451172255599772, 1.0402568447044462, 1.0256123778744273]


/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning:

Argument sub in putvarboundlist: Incorrect array format causing data to be copied

/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning:

Argument subj in putclist: Incorrect array format causing data to be copied

/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning:

Argument sub in putconboundlist: Incorrect array format causing data to be copied



### Check whether the solution is physically feasible
The semidefinite relaxation makes OPF convex, but also opens the door to:

1. Inexact solutions (i.e., WW is not rank-1)

2. Unrealistically cheap solutions that cannot occur in real networks.

Let's check our solution.

In [40]:
#Check the rank of W
eigvals = np.linalg.eigvalsh(W.value)
rank_W = np.sum(eigvals > 1e-6)  # numerical rank threshold
print("Rank of W:", rank_W)

# Check if the solution is feasible
for i in range(len(P_g_sdp)):
    if P_g_sdp[i] < min_p_mw[i]:
        print(f"Generation at bus {i} is below the minimum limit.")
    elif i > max_p_mw[i]:
        print(f"Generation at bus {i} is above the maximum limit.")

# Check if the voltage magnitudes are within limits
for i in range(n):
    if V_sdp[i]**2 < min_v_pu[i]:
        print(f"Voltage at bus {i} is below the minimum limit.")
    elif V_sdp[i]**2 > max_v_pu[i]:
        print(f"Voltage at bus {i} is above the maximum limit.")

# Check if the power balance equations are satisfied
for j in range(n):
    Pin_j = cp.trace(Phi[j] @ W).value * Sbase
    Qin_j = cp.trace(Psi[j] @ W).value * Sbase
    if j in gen_buses:
        idx = np.where(gen_buses == j)[0][0]  # Get generator index
        if Pin_j < min_p_mw[idx]:
            print(f"Power balance at bus {j} is below the minimum limit.")
        elif Pin_j > max_p_mw[idx]:
            print(f"Power balance at bus {j} is above the maximum limit.")
    # If bus is a load bus, check if the power balance is approximately equal to the load
    elif abs(Pin_j - p_load[j]) > 1e-7:
            print(f"Power balance at bus {j} does not match load.")



Rank of W: 1


### Benchmark using Classical OPF

In [55]:
pp.runopp(net)

#Print results
opf_cost = net.res_cost
print("OPF Cost ($/hr):", opf_cost)
P_g_opf = net.res_gen["p_mw"].values
print("OPF Real power generation (OPF):", P_g_opf)
Q_g_opf = net.res_gen["q_mvar"].values
print("OPF Reactive power generation (OPF):", Q_g_opf)
V_opf = net.res_bus["vm_pu"].values
print("OPF Voltage magnitudes (OPF):", V_opf)

gen vm_pu > bus max_vm_pu for gens [2 3]. Setting bus limit for these gens.


OPF Cost ($/hr): 8081.526613828872
OPF Real power generation (OPF): [3.67191563e+01 2.87425183e+01 7.57014046e-04 8.49478857e+00]
OPF Reactive power generation (OPF): [23.68509531 24.12687633 11.54531382  8.27296977]
OPF Voltage magnitudes (OPF): [1.06       1.04075293 1.01562489 1.01446054 1.01636232 1.05999866
 1.04634622 1.05999897 1.04369829 1.03913581 1.04600847 1.04481916
 1.03994764 1.02388767]


### Comparison

In [56]:
print("SDP OPF Optimal cost (EUR):", prob.value)
print("OPF Cost (EUR):", opf_cost*0.87)

SDP OPF Optimal cost (EUR): 4952.720791188003
OPF Cost (EUR): 7030.9281540311185


In [52]:
import plotly.graph_objects as go

# Generator power plot
gen_fig = go.Figure()
gen_fig.add_trace(go.Scatter(x=gen_buses, y=P_g_sdp, mode='markers+lines', name='SDP OPF Gen (MW)'))
gen_fig.add_trace(go.Scatter(x=gen_buses, y=P_g_opf, mode='markers+lines', name='OPF Gen (MW)'))
gen_fig.update_layout(title="Generator Real Power Comparison", xaxis_title="Generator Bus", yaxis_title="P (MW)")
gen_fig.show()

In [51]:
# Voltage Magnitude plot
gen_fig = go.Figure()
gen_fig.add_trace(go.Scatter(x=list(range(n)), y=V_sdp, mode='markers+lines', name='SDP OPF V (pu)'))
gen_fig.add_trace(go.Scatter(x=list(range(n)), y=V_opf, mode='markers+lines', name='OPF V (pu)'))
gen_fig.update_layout(title="Voltage Magnitude Comparison", xaxis_title="Bus", yaxis_title="V (pu)")
gen_fig.show()